In [18]:
# %tensorflow_version 1.x
# !pip install --upgrade opencv-python==3.4.2.17
import numpy as np
import tensorflow as tf
import tensorflow.keras
import tensorflow.keras.backend as K
# import os
from tensorflow.keras.datasets import fashion_mnist,mnist,cifar10
# import keras.backend as K
from tensorflow.keras.layers import Conv2D,Activation,BatchNormalization,UpSampling2D,Embedding,ZeroPadding2D, Input, Flatten, Dense, Reshape, LeakyReLU, Dropout,MaxPooling2D
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
from tensorflow.keras import regularizers

from tensorflow.keras.utils import Progbar
from keras.initializers import RandomNormal
import random
from sklearn.model_selection import train_test_split
# from keras.utils import np_utils
from tensorflow.keras import utils as np_utils



In [19]:
nb_classes = 10
batch_size = 128
maxepoches = 250
learning_rate = 0.1
lr_decay = 1e-6
lr_drop = 20
def lr_scheduler(epoch):
    return learning_rate * (0.5 ** (epoch // lr_drop))
reduce_lr = tf.keras.callbacks.LearningRateScheduler(lr_scheduler)

In [20]:
#Loading and splitting the dataset into train, validation and test


(X_Train, y_Train), (X_test, y_test) = cifar10.load_data()
X_train, X_val, y_train, y_val = train_test_split(X_Train, y_Train, test_size=0.20)
# convert y_train and y_test to categorical binary values 
Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_val = np_utils.to_categorical(y_val, nb_classes)
y_test = np_utils.to_categorical(y_test, nb_classes)

In [21]:
X_Train.shape

(50000, 32, 32, 3)

In [22]:
# Reshape them to batch_size, width,height,#channels
X_train = X_train.reshape(40000, 32, 32, 3)
X_val = X_val.reshape(10000, 32, 32, 3)
X_test = X_test.reshape(10000, 32, 32, 3)

X_train = X_train.astype('float32')
X_val = X_val.astype('float32')
X_test = X_test.astype('float32')

# Normalize the values
X_train /= 255
X_val /= 255
X_test /= 255

In [23]:
init=RandomNormal(mean=0,stddev=0.02)
input_shape = (32, 32, 3) # Input shape of each image
weight_decay = 0.0005
def build_model():
    # Build the network of vgg for 10 classes with massive dropout and weight decay as described in the paper.

    model = Sequential()

    model.add(Conv2D(64, (3, 3), padding='same',
                     input_shape=input_shape,kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.3))

    model.add(Conv2D(64, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())

    model.add(MaxPooling2D(pool_size=(2, 2),padding='same'))

    model.add(Conv2D(128, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.4))

    model.add(Conv2D(128, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())

    model.add(MaxPooling2D(pool_size=(2, 2),padding='same'))

    model.add(Conv2D(256, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.4))

    model.add(Conv2D(256, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.4))

    model.add(Conv2D(256, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())

    model.add(MaxPooling2D(pool_size=(2, 2),padding='same'))

    model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.4))

    model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.4))

    model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())

    model.add(MaxPooling2D(pool_size=(2, 2),padding='same'))

    model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.4))

    model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.4))

    model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())

    model.add(MaxPooling2D(pool_size=(2, 2),padding='same'))
    model.add(Dropout(0.5))

    model.add(Flatten())
    model.add(Dense(256,kernel_regularizer=regularizers.l2(weight_decay), name='dense_1'))
    model.add(Activation('relu'))
    model.add(BatchNormalization())

    model.add(Dropout(0.5))
    model.add(Dense(10, name='dense_2'))
    model.add(Activation('softmax'))
    return model
teacher = build_model()

sgd = SGD(lr=learning_rate, decay=lr_decay, momentum=0.9, nesterov=True)

teacher.compile(loss='categorical_crossentropy',optimizer=sgd, metrics=['accuracy'])


In [24]:
# teacher.fit(X_train,Y_train,batch_size=128,epochs=150,verbose=1,callbacks=[reduce_lr],validation_data=(X_val,Y_val))

In [25]:
teacher.load_weights("Cifar10_Teacher.h5")

In [26]:
# Y_test = np_utils.to_categorical(y_test, nb_classes)
loss, acc =teacher.evaluate(X_test, y_test, verbose=1)
loss, acc

313/313 [==============================] - 3s 10ms/step - loss: 0.8247 - accuracy: 0.8996


(0.834343671798706, 0.8992000222206116)

In [27]:
#Collect the dense vector from the previous layer output and store it in a different model
teacher_WO_Softmax = Model(teacher.input, teacher.get_layer('dense_1').output)

In [28]:
#Extracting dense representation from the teacher network
train_dense = teacher_WO_Softmax.predict(X_train)
val_dense = teacher_WO_Softmax.predict(X_val)

In [29]:
#Splitting the training dense vector among N students(in this case 2)
s1Train=train_dense[:,:64]
s2Train=train_dense[:,64:128]
s3Train=train_dense[:,128:192]
s4Train=train_dense[:,192:256]

s1Val=val_dense[:,:64]
s2Val=val_dense[:,64:128]
s3Val=val_dense[:,128:192]
s4Val=val_dense[:,192:256]

In [30]:
def define_model(name):
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(32, 32, 3), name=name))
    model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(0.2))
    model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(0.3))
    model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(0.4))
    model.add(Flatten())
    model.add(Dense(16, activation='relu', kernel_initializer='he_uniform'))
    model.add(Dropout(0.2))
    model.add(Dense(64, activation='relu', kernel_initializer='he_uniform',name='req'+name))

    model.compile(optimizer='nadam', loss='mse', metrics=['accuracy'])
    return model

student1 = define_model('s1')
student1.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
s1 (Conv2D)                  (None, 32, 32, 32)        896       
_________________________________________________________________
conv2d_45 (Conv2D)           (None, 32, 32, 32)        9248      
_________________________________________________________________
max_pooling2d_22 (MaxPooling (None, 16, 16, 32)        0         
_________________________________________________________________
dropout_36 (Dropout)         (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_46 (Conv2D)           (None, 16, 16, 64)        18496     
_________________________________________________________________
conv2d_47 (Conv2D)           (None, 16, 16, 64)        36928     
_________________________________________________________________
max_pooling2d_23 (MaxPooling (None, 8, 8, 64)         

TCN


In [31]:
# Testing with LR
s1=define_model("s1")
s2=define_model("s2")
s3=define_model("s3")
s4=define_model("s4")

opt=Adam(lr=0.0002, beta_1=0.9, beta_2=0.999, amsgrad=False)

In [32]:
s1.compile(loss='mse', optimizer=opt)
s2.compile(loss='mse', optimizer=opt)

s1.fit(X_train,s1Train,
          batch_size=256,
          epochs=80,
          verbose=1,
          validation_data=(X_val,s1Val))

s2.fit(X_train,s2Train,
          batch_size=256,
          epochs=60,
          verbose=1,
          validation_data=(X_val,s2Val))

Epoch 1/80
157/157 [==============================] - 3s 15ms/step - loss: 1.7822 - val_loss: 1.6569
Epoch 2/80
157/157 [==============================] - 2s 13ms/step - loss: 1.6473 - val_loss: 1.6235
Epoch 3/80
157/157 [==============================] - 2s 13ms/step - loss: 1.6114 - val_loss: 1.6001
Epoch 4/80
157/157 [==============================] - 2s 15ms/step - loss: 1.5845 - val_loss: 1.5294
Epoch 5/80
157/157 [==============================] - 2s 15ms/step - loss: 1.5239 - val_loss: 1.4756
Epoch 6/80
157/157 [==============================] - 3s 17ms/step - loss: 1.4934 - val_loss: 1.4499
Epoch 7/80
157/157 [==============================] - 2s 15ms/step - loss: 1.4732 - val_loss: 1.4413
Epoch 8/80
157/157 [==============================] - 3s 18ms/step - loss: 1.4643 - val_loss: 1.4281
Epoch 9/80
157/157 [==============================] - 3s 16ms/step - loss: 1.4519 - val_loss: 1.4093
Epoch 10/80
157/157 [==============================] - 3s 16ms/step - loss: 1.4391 - val_lo

157/157 [==============================] - 3s 15ms/step - loss: 1.7894 - val_loss: 1.6262
Epoch 2/60
157/157 [==============================] - 3s 16ms/step - loss: 1.6197 - val_loss: 1.6158
Epoch 3/60
157/157 [==============================] - 2s 14ms/step - loss: 1.6151 - val_loss: 1.6084
Epoch 4/60
157/157 [==============================] - 3s 17ms/step - loss: 1.6073 - val_loss: 1.6026
Epoch 5/60
157/157 [==============================] - 3s 17ms/step - loss: 1.6062 - val_loss: 1.5977
Epoch 6/60
157/157 [==============================] - 3s 16ms/step - loss: 1.6007 - val_loss: 1.5935
Epoch 7/60
157/157 [==============================] - 2s 15ms/step - loss: 1.5935 - val_loss: 1.5898
Epoch 8/60
157/157 [==============================] - 3s 17ms/step - loss: 1.5884 - val_loss: 1.5866
Epoch 9/60
157/157 [==============================] - 3s 18ms/step - loss: 1.5879 - val_loss: 1.5837
Epoch 10/60
157/157 [==============================] - 3s 16ms/step - loss: 1.5802 - val_loss: 1.5810


In [33]:
s3.compile(loss='mse', optimizer=opt)
s4.compile(loss='mse', optimizer=opt)

s3.fit(X_train,s3Train,
          batch_size=256,
          epochs=70,
          verbose=1,
          validation_data=(X_val,s3Val))

s4.fit(X_train,s4Train,
          batch_size=256,
          epochs=60,
          verbose=1,
          validation_data=(X_val,s4Val))

Epoch 1/70
157/157 [==============================] - 3s 15ms/step - loss: 1.6634 - val_loss: 1.5417
Epoch 2/70
157/157 [==============================] - 3s 18ms/step - loss: 1.5129 - val_loss: 1.4931
Epoch 3/70
157/157 [==============================] - 2s 16ms/step - loss: 1.4880 - val_loss: 1.4725
Epoch 4/70
157/157 [==============================] - 3s 17ms/step - loss: 1.4699 - val_loss: 1.4423
Epoch 5/70
157/157 [==============================] - 2s 13ms/step - loss: 1.4584 - val_loss: 1.4268
Epoch 6/70
157/157 [==============================] - 3s 18ms/step - loss: 1.4426 - val_loss: 1.4239
Epoch 7/70
157/157 [==============================] - 2s 14ms/step - loss: 1.4409 - val_loss: 1.4158
Epoch 8/70
157/157 [==============================] - 2s 15ms/step - loss: 1.4271 - val_loss: 1.4051
Epoch 9/70
157/157 [==============================] - 2s 14ms/step - loss: 1.4176 - val_loss: 1.3831
Epoch 10/70
157/157 [==============================] - 3s 17ms/step - loss: 1.4005 - val_lo

157/157 [==============================] - 2s 14ms/step - loss: 1.3211 - val_loss: 1.3004
Epoch 12/60
157/157 [==============================] - 2s 15ms/step - loss: 1.3129 - val_loss: 1.2888
Epoch 13/60
157/157 [==============================] - 2s 13ms/step - loss: 1.3085 - val_loss: 1.2856
Epoch 14/60
157/157 [==============================] - 2s 15ms/step - loss: 1.3087 - val_loss: 1.2805
Epoch 15/60
157/157 [==============================] - 2s 15ms/step - loss: 1.3011 - val_loss: 1.2769
Epoch 16/60
157/157 [==============================] - 2s 15ms/step - loss: 1.2918 - val_loss: 1.2717
Epoch 17/60
157/157 [==============================] - 3s 17ms/step - loss: 1.2915 - val_loss: 1.2543
Epoch 18/60
157/157 [==============================] - 2s 16ms/step - loss: 1.2722 - val_loss: 1.2335
Epoch 19/60
157/157 [==============================] - 2s 14ms/step - loss: 1.2618 - val_loss: 1.2236
Epoch 20/60
157/157 [==============================] - 3s 17ms/step - loss: 1.2529 - val_loss:

In [34]:
o1=s1.get_layer("reqs1").output
o2=s2.get_layer("reqs2").output
o3=s3.get_layer("reqs3").output
o4=s4.get_layer("reqs4").output
output=tensorflow.keras.layers.concatenate([o1,o2,o3,o4])

output=Activation('relu')(output)
output2=Dropout(0.5)(output) # For reguralization
output3=Dense(10,activation="softmax", name="d1")(output2)

mm4=Model([s1.get_layer("s1").input,s2.get_layer("s2").input,
           s3.get_layer("s3").input,s4.get_layer("s4").input], output3)
my_weights=teacher.get_layer('dense_2').get_weights()
mm4.get_layer('d1').set_weights(my_weights)

In [35]:
i=0
for l in mm4.layers[:len(mm4.layers)-2]:
    l.trainable=False
#     print(l)

mm4.compile(loss='categorical_crossentropy',
              optimizer=Adam(learning_rate=0.0002),
              metrics=['accuracy'])

# Without finetune
batch_size = 256
mm4_history=mm4.fit([X_train,X_train,X_train,X_train], Y_train,
          batch_size=batch_size,
          epochs=50,
          verbose=1,
          validation_data=([X_val,X_val,X_val,X_val], Y_val))

Epoch 1/50
157/157 [==============================] - 4s 23ms/step - loss: 0.9913 - accuracy: 0.7471 - val_loss: 0.7647 - val_accuracy: 0.8104
Epoch 2/50
157/157 [==============================] - 3s 19ms/step - loss: 0.7192 - accuracy: 0.8060 - val_loss: 0.6967 - val_accuracy: 0.8192
Epoch 3/50
157/157 [==============================] - 3s 19ms/step - loss: 0.6643 - accuracy: 0.8211 - val_loss: 0.6739 - val_accuracy: 0.8208
Epoch 4/50
157/157 [==============================] - 3s 18ms/step - loss: 0.6479 - accuracy: 0.8212 - val_loss: 0.6600 - val_accuracy: 0.8220
Epoch 5/50
157/157 [==============================] - 3s 19ms/step - loss: 0.6364 - accuracy: 0.8214 - val_loss: 0.6491 - val_accuracy: 0.8224
Epoch 6/50
157/157 [==============================] - 3s 18ms/step - loss: 0.5985 - accuracy: 0.8325 - val_loss: 0.6433 - val_accuracy: 0.8213
Epoch 7/50
157/157 [==============================] - 3s 18ms/step - loss: 0.6152 - accuracy: 0.8231 - val_loss: 0.6360 - val_accuracy: 0.8218